In [49]:
#!pip install geopandas
#!pip install geopy
#!pip install pgeocode

     |████████████████████████████████| 118 kB 3.8 MB/s eta 0:00:01


In [54]:
import pandas as pd
import setup
import matplotlib.pyplot as plt
import geopandas
import re
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from time import sleep
import seaborn as sns
%matplotlib inline

In [35]:
help(setup)

Help on module setup:

NAME
    setup

DATA
    __warningregistry__ = {'version': 30}

FILE
    /Users/elainatiller1/Desktop/SI330/final_project_330/setup.py




In [36]:
required_columns = ['Provider ID', 'Hospital Name', 'Address', 'City', 'State', 'ZIP Code', 'Hospital Type', \
               'Hospital Ownership', 'Emergency Services', 'Hospital overall rating', \
               'Patient experience national comparison', 'Mortality national comparison', \
               'Effectiveness of care national comparison']

In [37]:
crime_data = pd.read_csv('crime_data_w_population_and_crime_rate.csv')
hospitals = pd.read_csv('Hospital.csv', encoding='Windows-1252', usecols=required_columns)

In [38]:
hospitals.head()

,Provider ID,Hospital Name,Address,City,State,ZIP Code,Hospital Type,Hospital Ownership,Emergency Services,Hospital overall rating,Mortality national comparison,Patient experience national comparison,Effectiveness of care national comparison
0,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Acute Care Hospitals,Government - Hospital District or Authority,Yes,3,Same as the national average,Below the national average,Same as the national average
1,10005,MARSHALL MEDICAL CENTER SOUTH,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,Acute Care Hospitals,Government - Hospital District or Authority,Yes,3,Below the national average,Same as the national average,Same as the national average
2,10006,ELIZA COFFEE MEMORIAL HOSPITAL,205 MARENGO STREET,FLORENCE,AL,35631,Acute Care Hospitals,Government - Hospital District or Authority,Yes,2,Below the national average,Below the national average,Same as the national average
3,10007,MIZELL MEMORIAL HOSPITAL,702 N MAIN ST,OPP,AL,36467,Acute Care Hospitals,Voluntary non-profit - Private,Yes,2,Same as the national average,Same as the national average,Below the national average
4,10008,CRENSHAW COMMUNITY HOSPITAL,101 HOSPITAL CIRCLE,LUVERNE,AL,36049,Acute Care Hospitals,Proprietary,Yes,3,Same as the national average,Not Available,Same as the national average


In [39]:
crime_data.head()

,county_name,crime_rate_per_100000,index,EDITION,PART,IDNO,CPOPARST,CPOPCRIM,AG_ARRST,AG_OFF,...,RAPE,ROBBERY,AGASSLT,BURGLRY,LARCENY,MVTHEFT,ARSON,population,FIPS_ST,FIPS_CTY
0,"St. Louis city, MO",1791.995377,1,1,4,1612,318667,318667,15,15,...,200,1778,3609,4995,13791,3543,464,318416,29,510
1,"Crittenden County, AR",1754.914968,2,1,4,130,50717,50717,4,4,...,38,165,662,1482,1753,189,28,49746,5,35
2,"Alexander County, IL",1664.700485,3,1,4,604,8040,8040,2,2,...,2,5,119,82,184,12,2,7629,17,3
3,"Kenedy County, TX",1456.310680,4,1,4,2681,444,444,1,1,...,3,1,2,5,4,4,0,412,48,261
4,"De Soto Parish, LA",1447.402430,5,1,4,1137,26971,26971,3,3,...,4,17,368,149,494,60,0,27083,22,31


In [40]:
hospitals.isna().sum()

Provider ID                                  0
Hospital Name                                0
Address                                      0
City                                         0
State                                        0
ZIP Code                                     0
Hospital Type                                0
Hospital Ownership                           0
Emergency Services                           0
Hospital overall rating                      0
Mortality national comparison                0
Patient experience national comparison       0
Effectiveness of care national comparison    0
dtype: int64

In [41]:
len(hospitals.index)

4812

In [42]:
hospitals['Patient experience national comparison'].value_counts()

Not Available                   1369
Above the national average      1214
Same as the national average    1148
Below the national average      1081
Name: Patient experience national comparison, dtype: int64

In [43]:
hospitals['Mortality national comparison'].value_counts()

Same as the national average    2719
Not Available                   1352
Above the national average       400
Below the national average       341
Name: Mortality national comparison, dtype: int64

In [44]:
hospitals['Effectiveness of care national comparison'].value_counts()

Same as the national average    3238
Not Available                   1202
Below the national average       242
Above the national average       130
Name: Effectiveness of care national comparison, dtype: int64

In [55]:
from geopy.geocoders import Photon
geolocator = Photon(user_agent="Final Project1", timeout=10)

In [56]:
hospitals['ZIP Code'] = hospitals['ZIP Code'].astype(str)

In [48]:
hospitals['full_address'] = hospitals['Address'] + ' ' + hospitals['City'] + ' ' + hospitals['State'] + ' ' + \
                            hospitals['ZIP Code']

In [49]:
hospitals['full_address'].tail(20)

4792                1975 ALPHA STE 100 ROCKWALL TX 75087
4793          555 CREEKSIDE XING, NEW BRAUNFELS TX 78130
4794                  9201 PINECROFT SHENANDOAH TX 77380
4795      12950 EAST FREEWAY, SUITE 100 HOUSTON TX 77015
4796      3101 NORTH TARRANT PARKWAY FORT WORTH TX 76177
4797        11100 SHADOW CREEK PARKWAY PEARLAND TX 77584
4798     12500 SOUTH FREEWAY SUITE 100 BURLESON TX 76028
4799              810 W HIGHWAY 71 MARBLE FALLS TX 78654
4800                    1626 W BAKER RD BAYTOWN TX 77521
4801    500 WEST  4TH STREET , 4TH FLOOR ODESSA TX 79761
4802      5330 NORTH LOOP 1604 WEST SAN ANTONIO TX 78249
4803                       400 YOUENS DR WEIMAR TX 78962
4804                1017 S TRAVIS AVE CLEVELAND TX 77327
4805      3200 NORTH TARRANT PARKWAY FORT WORTH TX 76177
4806              707 N WALDRIP ST GRAND SALINE TX 75140
4807                    9922 LOUETTA RD HOUSTON TX 77070
4808                  FAGAALU VILLAGE PAGO PAGO AS 96799
4809     85O GOV CARLOS G CAMAC

In [57]:
geolocator.geocode("1975 ALPHA STE 100 ROCKWALL TX 75087")

Location(Baylor Emergency Medical Center, 1975, Alpha Drive, 75087, Alpha Drive, Rockwall, TX, United States, (32.9142461, -96.44447344031661, 0.0))

In [ ]:
geolocator.geocode("Alexander County, IL")

In [ ]:
test_hospitals = hospitals.head(5)

In [ ]:
import pgeocode
nomi = pgeocode.Nominatim('US')

In [ ]:
nomi.query_postal_code("06482")

In [ ]:
nomi.query_postal_code("78130")

# Getting Latitude/Longitude for hospitals and crime data

In [62]:
#Inputs: string as county to query geolocator with (St. Louis city, MO)
#Output: tuple wit zipcode (int) and str of lat/long
def zip_and_lat_long_for_crime(county):
    if type(county) != type('string'):
        print('Inputs were not correct. Ensure text is a string.')
    else:
        location = geolocator.geocode(county)
        if(location == None):
            print(location)
            return "000000", "(Na,Na)"
        return location[0].split(', ')[1], location[1]

#Inputs: string address to query geolocator with ex:("1975 ALPHA STE 100 ROCKWALL TX 75087")
#Output: str of lat/long
def lat_long_for_hospital(address):
    if type(address) != type('string'):
        print('Inputs were not correct. Ensure text is a string.')
    else:
        location = geolocator.geocode(address)
        if(location != None):
            return location[1]

            

In [63]:
crime_data['zip_code'], crime_data['lat_long'] = zip(*crime_data['county_name'].map(zip_and_lat_long_for_crime))


None
None
None
None
None
None


In [64]:
crime_data.head()

,county_name,crime_rate_per_100000,index,EDITION,PART,IDNO,CPOPARST,CPOPCRIM,AG_ARRST,AG_OFF,...,AGASSLT,BURGLRY,LARCENY,MVTHEFT,ARSON,population,FIPS_ST,FIPS_CTY,zip_code,lat_long
0,"St. Louis city, MO",1791.995377,1,1,4,1612,318667,318667,15,15,...,3609,4995,13791,3543,464,318416,29,510,70380,"(29.6993748, -91.20677)"
1,"Crittenden County, AR",1754.914968,2,1,4,130,50717,50717,4,4,...,662,1482,1753,189,28,49746,5,35,Arkansas,"(35.2505701, -90.3192299)"
2,"Alexander County, IL",1664.700485,3,1,4,604,8040,8040,2,2,...,119,82,184,12,2,7629,17,3,Illinois,"(37.1801529, -89.3502834)"
3,"Kenedy County, TX",1456.310680,4,1,4,2681,444,444,1,1,...,2,5,4,4,0,412,48,261,Mallory Street,"(27.221984, -97.79136)"
4,"De Soto Parish, LA",1447.402430,5,1,4,1137,26971,26971,3,3,...,368,149,494,60,0,27083,22,31,Louisiana,"(31.901498, -93.670989)"


In [65]:
hospitals['lat_long'] = hospitals['full_address'].map(lat_long_for_hospital)


In [66]:
hospitals.head()

,Provider ID,Hospital Name,Address,City,State,ZIP Code,Hospital Type,Hospital Ownership,Emergency Services,Hospital overall rating,Mortality national comparison,Patient experience national comparison,Effectiveness of care national comparison,full_address,lat_long
0,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Acute Care Hospitals,Government - Hospital District or Authority,Yes,3,Same as the national average,Below the national average,Same as the national average,1108 ROSS CLARK CIRCLE DOTHAN AL 36301,"(31.216402000000002, -85.36564064472296)"
1,10005,MARSHALL MEDICAL CENTER SOUTH,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,Acute Care Hospitals,Government - Hospital District or Authority,Yes,3,Below the national average,Same as the national average,Same as the national average,2505 U S HIGHWAY 431 NORTH BOAZ AL 35957,None
2,10006,ELIZA COFFEE MEMORIAL HOSPITAL,205 MARENGO STREET,FLORENCE,AL,35631,Acute Care Hospitals,Government - Hospital District or Authority,Yes,2,Below the national average,Below the national average,Same as the national average,205 MARENGO STREET FLORENCE AL 35631,"(34.7946245, -87.68303582998244)"
3,10007,MIZELL MEMORIAL HOSPITAL,702 N MAIN ST,OPP,AL,36467,Acute Care Hospitals,Voluntary non-profit - Private,Yes,2,Same as the national average,Same as the national average,Below the national average,702 N MAIN ST OPP AL 36467,"(31.2928787, -86.25490030241093)"
4,10008,CRENSHAW COMMUNITY HOSPITAL,101 HOSPITAL CIRCLE,LUVERNE,AL,36049,Acute Care Hospitals,Proprietary,Yes,3,Same as the national average,Not Available,Same as the national average,101 HOSPITAL CIRCLE LUVERNE AL 36049,"(31.6937493, -86.26470160380265)"


# Haversine Distance - finding closest hospital 

In [94]:
#Inputs: str of state
#Output: df of hospitals + (lat,long) for that state
def return_matching_state_df(state):
    hospitals_for_state = hospitals[hospitals['State']==state]
    hospitals_for_state=hospitals_for_state[['Hospital Name','lat_long']]
    return hospitals_for_state

#Inputs: series frome crime data
#Output: str of Hospital Name from hospital dataset 
def find_closest_hospital(series):
    lat_long = series['lat_long']
    state = series['county_name'].split(', ')[1]
    df = return_matching_state_df(state)
    lat_long_list = df.lat_long.tolist()
    #https://readthedocs.org/projects/pgeo
    print(lat_long_list)

    distances = pgeocode.haversine_distance([lat_long], lat_long_list)
    return min(distances)

In [95]:
crime_data['closest_hospital'] = crime_data.apply(find_closest_hospital, axis=1)

[(37.03651775, -94.509838379283), (38.943545549999996, -92.7403022780093), (38.7808818, -90.48468694474604), None, (38.69897455, -93.22100141339578), (38.55854415, -92.2220520580006), None, (37.95219915, -91.78106352489056), (38.6454887, -90.4460795385316), (40.19310445, -92.5889630445565), None, (37.31578615, -91.96077202601234), (39.71054465, -91.44819025486223), None, None, (38.2486852, -94.34286596231128), (37.144411250000005, -93.2803691826818), (38.56597585, -92.18142175318368), (39.0857029, -94.57481611350138), (40.32509245, -94.87660710787983), (38.550413649999996, -91.00302859410569), (39.73039045, -94.2174126366219), (37.68130705, -92.63406483343294), (37.8303372, -94.36076668599668), None, (39.1725433, -91.8779415), (37.1779865, -93.27414378114038), (38.94891115, -92.3157216364498), (36.23756695, -89.74067909750396), (39.42545115, -92.41414562608827), (38.50613535, -90.38330256066584), (36.738344600000005, -91.87529342276618), (36.629604349999994, -90.82779184619405), (38.52

TypeError: loop of ufunc does not support argument 0 of type tuple which has no callable radians method

# group by zipcode to find avg hospital rating 

In [96]:
hospitals['ZIP Code'].value_counts()

77030    6
70508    5
76104    5
73120    4
19104    4
        ..
75401    1
23505    1
41017    1
14642    1
68845    1
Name: ZIP Code, Length: 4416, dtype: int64

In [ ]:
crime_data['zip_code'].value_counts()

In [ ]:
crime_rate_per_100000_by_zip = crime_data.groupby('zip_code')['crime_rate_per_100000']

# Make scatterplot with  zip codes on X axis and crime_rate_per_100000 on Y axis 

In [ ]:
#x axis will be zip codes ranging from lowest zip code possible(00501) 
#to highest zip code possible (99950)

In [ ]:
titlefont = {'size': 16, 'fontname':'Comic Sans MS'}

sns.lineplot(x="zip codes", y="crime rate per 100000", data=)


In [ ]:
# Make barchart with 3 farthest/closest hopsitals to county vs. crime_rate_per_100000 on Y axis 

In [ ]:
three_closest_hospitals = 

## Problem Statement
My goal with this notebook is to use to answer the question: *is class positively correlated with survival on the ship Titanic?*

## Data

To answer this question I will use a dataset from Kaggle called titanic 
(link here: [titanic dataset](https://www.kaggle.com/c/titanic)). I collected this by downloading the csv. The data is stored in a csv format. There are 891 rows and 12 columns. The total size of the file is 59 kB. 

## Data cleaning and manipulation

To address my central question I did the following data cleaning steps:
    ....
    
This are all contained in the file data_cleaning_functions.py. 

## Analysis

To present my analysis I first consider a table of survival rates for each class. Here I can see that the proportion of passengers in the first class who survived is much higher than for the second and third classes. 

However, it is possible that this gap is not trustworthy if there is very little data in one of these classes. So I will also plot survival and include the size of the category in the plot. 

In the plot although we can see there is more data in the third class the amount of data in the first class is not so small that I am concerned. I do think there is a statistically valid relationship between class and survival. However, to be triple sure, I'll check the correlation coefficient between plcass and survival rate and check that it is statistically significant. 

Surprisingly, although the correlation coefficient is very strong negative, it is not statistically signifanct (probably because I only have three values and have lost all information about the size of the data). 


Now I see a weaker but much more significant relationship between the two variables. However, it the pearson correlation coefficient isn't really set up for this kind of comparison, so in future work I will use a better measure. 

## Take-aways
For now, I am convinced that as class goes up from 3 -> 1 there is an increase in survival rates. While the relationship is strong and seems to be significant, I will test the significant levels more thoroughly in future work.  

## References 

Include all references to sources for the code, sources for any packages or methods you used and a guide to all of the files you wrote and are including. 